<a href="https://colab.research.google.com/github/CarlosAudre/CarlosAudre/blob/main/streamlit_dashboards.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit pyngrok
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -q ngrok-stable-linux-amd64.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.5 MB/s eta 0:00:00


In [ ]:
pip install psycopg2-binary sqlalchemy pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 22.8 MB/s eta 0:00:00


In [ ]:

%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sqlalchemy import create_engine

usuario = "****"
senha = "****"
host = "****"
porta = "5432"
banco = "postgres"

url = f"postgresql://{usuario}:{senha}@{host}:{porta}/{banco}"

engine = create_engine(url)

tabela = "Fact_Transacoes"

df_fact     = pd.read_sql("SELECT * FROM fact_transacoes", engine)
df_tempo    = pd.read_sql("SELECT * FROM dim_tempo",     engine)
df_pais     = pd.read_sql("SELECT * FROM dim_pais",      engine)
df_produto  = pd.read_sql("SELECT * FROM dim_produto",   engine)
df_transp   = pd.read_sql("SELECT * FROM dim_transporte",engine)
df_moeda    = pd.read_sql("SELECT * FROM dim_moeda",     engine)

df = (df_fact
      .merge(df_tempo,    left_on="fk_tempo",        right_on="id_tempo")
      .merge(df_pais,     left_on="fk_pais_origem",  right_on="id")
      .merge(df_pais,     left_on="fk_pais_destino", right_on="id", suffixes=("_orig","_dest"))
      .merge(df_produto,  left_on="fk_produto",      right_on="id")
      .merge(df_transp,   left_on="fk_transporte",   right_on="id")
      .merge(df_moeda,    left_on="fk_moeda_origem", right_on="id_moeda", suffixes=("","_moeda_dest")))


st.set_page_config(layout='wide')
st.title("Gestão Global de Transações — StarComex")

df['data'] = pd.to_datetime(df['data'],format="%Y-%m-%d")
df['ano']  = df['data'].dt.year

def formataNumero(valor, prefixo=''):
    for unidade in ['', 'mil']:
        if valor < 1000:
            return f'{prefixo} {valor:.2f} {unidade}'
        valor /= 1000
    return f'{prefixo} {valor:.2f} milhões'

#Tabelas

df_export = df[df["tipo_transacao"] == "EXPORT"]
df_export = df_export.groupby("nome_pais_orig")[["valor_monetario"]].sum()
#----------------------------------------------------------------------------------------------------------------------------------------

df_produto_volume = df.groupby("descricao_x")[["quantidade"]].sum().sort_values("quantidade", ascending=False)

#----------------------------------------------------------------------------------------------------------------------------------------
df_origem = df.groupby(['bloco_economico_orig', 'ano'])['valor_monetario'].sum().reset_index()
df_origem.rename(columns={'bloco_economico_orig' : 'bloco'}, inplace = True)

df_destino = df.groupby(['bloco_economico_dest', 'ano'])['valor_monetario'].sum().reset_index()
df_destino.rename(columns={'bloco_economico_dest' : 'bloco'}, inplace = True)

df_total = pd.concat([df_origem, df_destino])
df_total = df_total.groupby(['bloco', 'ano'])['valor_monetario'].sum().reset_index()

#----------------------------------------------------------------------------------------------------------------------------------------
df_parceiros = df.groupby(['nome_pais_orig', 'nome_pais_dest'])[['valor_monetario']].sum().reset_index()
df_parceiros = df_parceiros.sort_values(['nome_pais_orig', 'valor_monetario'], ascending=[True, False])

#----------------------------------------------------------------------------------------------------------------------------------------

df_variacao_taxas_cambio = (
    df
    .groupby('ano')
    .agg(
        quantidade_total = ('quantidade', 'sum'),
        taxa_media        = ('taxa_cambio', 'mean')
    )
    .reset_index()
)

#----------------------------------------------------------------------------------------------------------------------------------------
df_transporte = df.groupby("descricao_y")["valor_monetario"].sum().reset_index()
df_transporte = df_transporte.sort_values("valor_monetario", ascending=False)

#----------------------------------------------------------------------------------------------------------------------------------------
df_resumo_valores = (
    df
    .groupby(["ano", "tipo_transacao"])["valor_monetario"]
    .sum()
    .reset_index()
    .sort_values("ano")
)
#----------------------------------------------------------------------------------------------------------------------------------------

#Gráficos

df_pie = df_export.reset_index()
fig_df_export = px.pie(
    df_pie,
    names='nome_pais_orig',
    values='valor_monetario',
    title='Países que mais exportam'
)
#----------------------------------------------------------------------------------------------------------------------------------------

fig_df_produtos_volume = px.bar(df_produto_volume,
                                text_auto = True,
                                title = 'Produtos com maior volume de exportação/importação'
                                )
fig_df_produtos_volume.update_layout(
    xaxis_title='Produto',
    yaxis_title='Quantidade'
)
#----------------------------------------------------------------------------------------------------------------------------------------

fig_evolucao_bloco = px.line(df_total,
              x='ano',
              y='valor_monetario',
              color='bloco',
              title='Evolução do Comércio por Bloco Econômico ao Longo do Tempo')

fig_evolucao_bloco.update_layout(
    xaxis_title = 'Ano',
    yaxis_title = 'Valor monetário'
)



#----------------------------------------------------------------------------------------------------------------------------------------

fig_parceiros = px.bar(
    df_parceiros,
    x='valor_monetario',
    y='nome_pais_orig',
    color='nome_pais_dest',
    orientation='h',
    title='Principais Parceiros Comerciais por País',
    text_auto=True
)
fig_parceiros.update_layout(
    xaxis_title='Valor Monetário',
    yaxis_title='País de Origem',
    legend_title='País de Destino'
)

#----------------------------------------------------------------------------------------------------------------------------------------

# 1) barras de quantidade
fig_variacao_taxa = px.bar(
    df_variacao_taxas_cambio,
    x='ano',
    y='quantidade_total',
    labels={
        'ano': 'Período',
        'quantidade_total': 'Quantidade transacionada'
    },

)

# 2) linha da taxa média no eixo y2
fig_variacao_taxa.add_trace(
    go.Scatter( #Cria uma linha com marcadores
        x=df_variacao_taxas_cambio['ano'],
        y=df_variacao_taxas_cambio['taxa_media'],
        mode='lines+markers',
        name='Taxa de câmbio média',
        marker_color='firebrick',
        yaxis='y2'
    )
)

# 3) configura os dois eixos Y
fig_variacao_taxa.update_layout(
    title='Variação da Taxa de Câmbio x Volume de Comércio',
    xaxis_title='Período',
    yaxis=dict(title='Quantidade total'),
    yaxis2=dict(
        title='Taxa de câmbio média',
        overlaying='y',
        side='right'
    ),
    legend=dict(x=0.1, y=1.1, orientation='h'),
    margin=dict(t=80, b=40),
)

#----------------------------------------------------------------------------------------------------------------------------------------

fig_transporte_pie = px.pie(
    df_transporte,
    names="descricao_y",
    values="valor_monetario",
    title="Distribuição dos Meios de Transporte nas Transações Globais"
)

#----------------------------------------------------------------------------------------------------------------------------------------

fig_total_exp_imp_ano = px.bar(
    df_resumo_valores,
    x="ano",
    y="valor_monetario",
    color="tipo_transacao",
    barmode="group",
    text_auto=True,
    labels={
        "ano": "Ano",
        "valor_monetario": "Valor Total (R$)",
        "tipo_transacao": "Tipo de Transação"
    },
    title="Valor Total de Importação e Exportação por Ano (Todos os Países)"
)

fig_total_exp_imp_ano.update_layout(yaxis_title="Valor Total (R$)")

#----------------------------------------------------------------------------------------------------------------------------------------


#Visual
coluna1, coluna2 = st.columns(2)
with coluna1:
  st.metric('Valor total', formataNumero(df['valor_monetario'].sum(), 'R$'))
  st.plotly_chart(fig_df_export, use_container_width=True)
  st.plotly_chart(fig_evolucao_bloco, use_container_width=True)
  st.plotly_chart(fig_variacao_taxa, use_container_width=True)
  st.plotly_chart(fig_total_exp_imp_ano, use_container_width=True)

with coluna2:
  st.metric('Quantidade total', formataNumero(df['quantidade'].sum(), 'R$'))
  st.plotly_chart(fig_df_produtos_volume, use_container_width=True)
  st.plotly_chart(fig_parceiros, use_container_width=True)
  st.plotly_chart(fig_transporte_pie, use_container_width=True)


st.dataframe(df)


Writing app.py


In [ ]:
!pip install streamlit pandas plotly requests pyngrok

In [ ]:
!./ngrok authtoken 2v3ENm6eOvrKNOCcOuM7AnCgGID_3vscNoTtAg7PC6zNgwctj

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!streamlit run app.py --server.port 8501 &>/dev/null &

In [ ]:
from pyngrok import conf, ngrok

conf.get_default().auth_token = "2v3ENm6eOvrKNOCcOuM7AnCgGID_3vscNoTtAg7PC6zNgwctj"
public_url = ngrok.connect(8501)
print(public_url)

ERROR:pyngrok.process.ngrok:t=2025-04-21T18:31:54+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n"
ERROR:pyngrok.process.ngrok:t=2025-04-21T18:31:54+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok st

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n.